In [5]:
search_for = 122
start_from = 0
threads = 10
thread = 1
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.0041773319244384766
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 25
all cases: 436374755
type: [1, 1, 1, 1, 122] 122
cases of this type: 221533456
100000 22.243127096060384
200000 39.13294191614105
300000 39.11777182190808
400000 38.50369350272087
500000 35.39082250599364
600000 38.91217305291305
700000 37.57835648227706
800000 38.73853880397519
900000 26.71788856585338
1000000 21.835424690664155
1100000 34.7971351228203
1200000 30.759875604009657
1300000 33.97312774951474
1400000 31.904436859517475
1500000 32.817546759144044
1600000 30.152887653546742
1700000 33.150925882190556
1800000 28.681593917844637
1900000 13.93685326058688
2000000 32.75032500956345
2100000 29.72014048675893
2200000 31.934337556690174
2300000 30.891910263369024
2400000 30.3727709654954
2500000 31.244437388473564
2600000 29.126447372762325
2700000 30.987280891582152
2800000 12.679110270783612
2900000 29.008641741616024
3000000 31.46

29500000 29.952393886454814
29600000 30.87308873969606
29700000 28.490740569106713
29800000 26.70716118136479
29900000 27.94830759272358
30000000 11.803249093609493
30100000 26.23645661753245
30200000 28.339974347299805
30300000 27.59980427239648
30400000 30.22524133737634
30500000 29.68557823183164
30600000 30.5332242834426
30700000 15.393907154311522
30800000 28.626556333938538
30900000 31.827704025759115
31000000 30.521088320603702
31100000 31.92726689647893
31200000 15.410939082844932
31300000 30.62365800662869
31400000 31.979627684751573
31500000 29.69147428887729
31600000 30.338725039093056
31700000 27.056159465549378
31800000 15.454202719373026
31900000 25.24477475862822
32000000 26.869121796781283
32100000 31.502229119104044
32200000 31.80872347866763
32300000 31.042258498609844
32400000 34.83833829531232
32500000 15.25209939762446
32600000 33.81567749840998
32700000 31.56079502357399
32800000 30.124990483103023
32900000 30.565318290753662
33000000 14.416229326547697
33100000 3

59200000 15.256713632066164
59300000 14.96776263499728
59400000 15.935415994181861
59500000 8.717565461015678
59600000 23.59200400993773
59700000 17.463393759647083
59800000 12.503658926780664
59900000 18.356144935450853
60000000 13.298716786574214
60100000 12.87393769653805
60200000 13.596363542656182
60300000 15.990423070274273
60400000 11.96801132633468
60500000 6.909975823045393
60600000 16.960419367245393
60700000 19.39966220662066
60800000 20.494436099279305
60900000 5.219710513197568
61000000 12.283714390271198
61100000 19.863002280138286
61200000 14.42063836616762
61300000 11.82876596119308
61400000 12.787233188974419
61500000 12.785392591848938
61600000 13.9957577534733
61700000 15.055717875716924
61800000 16.632421213245404
61900000 16.721338564013404
62000000 15.527038491924703
62100000 19.12311261158081
62200000 12.512781148164247
62300000 6.292275895684679
62400000 10.95636421878286
62500000 16.417928996657253
62600000 19.593630937199283
62700000 6.878251176104678
62800000

88900000 25.072563133187163
89000000 26.03289172803822
89100000 26.502790519402893
89200000 26.841755033508687
89300000 24.157013446732652
89400000 25.574032494623122
89500000 24.502963812247774
89600000 24.985626773821345
89700000 20.86109815292099
89800000 16.127610585914972
89900000 14.00772472321248
90000000 17.47691932199873
90100000 13.376926801389619
90200000 22.894716268709182
90300000 25.682459926827086
90400000 23.700023492123425
90500000 25.10417535267054
90600000 23.822706224689842
90700000 26.15185741210828
90800000 28.942574935662837
90900000 25.261426841601303
91000000 26.11080891991991
91100000 23.152538937832325
91200000 23.438178315123157
91300000 23.10792725498924
91400000 23.03355106832837
91500000 23.125788950245045
91600000 11.63017116782969
91700000 16.86676622918114
91800000 15.510934003203584
91900000 13.574383001759204
92000000 22.11841267247512
92100000 22.84299289844065
92200000 22.927480540693328
92300000 23.052772400534074
92400000 23.320061863802575
92500

118000000 31.539790407658188
118100000 28.363130601592538
118200000 24.9946723448882
118300000 24.475227726384812
118400000 22.95293012515946
118500000 25.26364713870287
118600000 24.629441890898686
118700000 23.90820945550114
118800000 22.73297329679663
118900000 15.048153168894977
119000000 3.794569668162306
119100000 16.54468848582175
119200000 23.541529024740374
119300000 24.734760607645274
119400000 24.246474731161552
119500000 24.498065112077754
119600000 23.904542441843333
119700000 24.688223887489894
119800000 25.379960048090975
119900000 25.811744456052214
120000000 21.80902708249785
120100000 21.51898539046643
120200000 19.62730980017533
120300000 19.7815961691817
120400000 20.590346279783034
120500000 20.519065175148366
120600000 18.61583162702577
120700000 13.520544495777179
120800000 3.650456927016576
120900000 10.887897060159633
121000000 20.586715669241244
121100000 20.204703208172965
121200000 19.71460872731287
121300000 20.230018004063226
121400000 20.726270661427378
1

...

319900000 4.292649510047101
320000000 6.135389648978064
320100000 6.1668903282851915
320200000 5.690209449547141
320300000 6.190079756394267
320400000 5.579066144198435
320500000 2.537127412893673
320600000 2.8686702358723384
320700000 5.984180467782193
320800000 5.8370428799379575
320900000 6.41602311358122
321000000 6.794939249973751
321100000 6.12698494086212
321200000 6.559270366436452
321300000 6.187288251922494
321400000 6.51491199883254
321500000 6.018933472130795
321600000 2.9272254978142977
321700000 5.2220969967102935
321800000 5.849614749423752
321900000 5.684727616912687
322000000 5.819945072644909
322100000 5.53927688896346
322200000 5.319626229656775
322300000 3.37864686538785
322400000 1.7835102663160363
322500000 6.124649047765775
322600000 6.018218697203507
322700000 6.137603095095675
322800000 5.885787324268941
322900000 6.081000408733898
323000000 6.453145079730213
323100000 5.893027484579212
323200000 6.8040942779575255
323300000 6.239251771744798
323400000 2.826885

348100000 3.733646616447429
348200000 2.600184238562584
348300000 2.642895288288832
348400000 3.4677093615688284
348500000 2.7334953148031333
348600000 3.6682080954519445
348700000 4.077388414634175
348800000 3.8270506919808684
348900000 3.2099465174535475
349000000 2.6106078488286997
349100000 2.816690922028512
349200000 2.6608035960977015
349300000 3.379415024525745
349400000 2.2834650783114334
349500000 3.0031922084898386
349600000 3.6107441512228946
349700000 3.837821062752018
349800000 2.9720825406013662
349900000 3.068760111062752
350000000 3.4056689295406937
350100000 2.296119620910128
350200000 3.4235490485366613
350300000 1.89540471640416
350400000 3.8169825768278605
350500000 3.886082268504802
350600000 3.922370597702166
350700000 2.144551095285707
350800000 3.6716569681896205
350900000 3.048313196300149
351000000 1.3669811997013424
351100000 3.0193628494753804
351200000 2.4566090568997465
351300000 3.8114063859121763
351400000 4.124837426042852
351500000 3.7753297699730988
3

376600000 1.0434190014007918
376700000 1.3830459443087082
376800000 1.5654617202617858
376900000 1.9677275778724486
377000000 1.3261146761119367
377100000 0.9808691326821877
377200000 1.3060429572569532
377300000 0.7946875766162972
377400000 1.2316852173168757
377500000 1.0000819071417284
377600000 1.691749983848979
377700000 1.4222943082764548
377800000 1.3107465485131906
377900000 1.407420161913991
378000000 0.9077100125985444
378100000 1.2324476718385617
378200000 0.9419825723021924
378300000 1.923268061138067
378400000 0.9867811856914362
378500000 1.2929594169753988
378600000 1.4421714476612078
378700000 1.6685610169239573
378800000 1.6540052542898953
378900000 0.9084151894941993
379000000 1.221288010776109
379100000 1.2117115310866633
379200000 1.2509134087592
379300000 1.0502946085074578
379400000 1.7455596427719098
379500000 1.6759922449841267
379600000 1.461087008270449
379700000 1.3263579120960831
379800000 0.8903827380381724
379900000 1.2249133359962339
380000000 0.7600181122

403900000 0.43570551538883645
404000000 0.42962792027816177
404100000 0.781894132576757
404200000 0.7784269716563886
404300000 0.6688855680570702
404400000 0.4364060897940762
404500000 0.5067205112108687
404600000 0.6408939421122074
404700000 0.62023858656418
404800000 0.46156510272918805
404900000 0.3732041638465524
405000000 0.5907711366862125
405100000 0.8337988648970028
405200000 0.5948987413039605
405300000 0.2990667264702022
405400000 0.5245721890898579
405500000 0.6183297318698864
405600000 0.6035934963919786
405700000 0.4772115020640923
405800000 0.32753444468942283
405900000 0.5693911431072023
406000000 0.6383929370604985
406100000 0.5860672635125717
406200000 0.298710281276236
406300000 0.5296234936262635
406400000 0.5972743568356534
406500000 0.6785986389197475
406600000 0.6110882117254965
406700000 0.360846176574111
406800000 0.6313980727023516
406900000 0.43094171093644695
407000000 0.11599870259174704
407100000 0.14267128245657681
407200000 0.11596785374534131
407300000 0

430100000 0.10011048165366053
430200000 0.06733369684917398
430300000 0.09257074731559223
430400000 0.061305013441870615
430500000 0.0882307329418328
430600000 0.06996259304471479
430700000 0.0811570203761657
430800000 0.07720092090647751
430900000 0.0617278048444589
431000000 0.08117841529646516
431100000 0.055324915126926374
431200000 0.07882051832595136
431300000 0.055446814867258075
431400000 0.07359653200869427
431500000 0.05788174110601015
431600000 0.06618641695753733
431700000 0.0676853877333535
431800000 0.05448094852931301
431900000 0.07084050553584098
432000000 0.048587754419757254
432100000 0.06992576300721368
432200000 0.04992985883676012
432300000 0.05929193190216356
432400000 0.05096760771531198
432500000 0.05074495685482025
432600000 0.05612413912121124
432700000 0.04274862974559599
432800000 0.05458556671947241
432900000 0.04435051944504181
433000000 0.050126018196937114
433100000 0.04467581421441833
433200000 0.044270293374203976
433300000 0.04215936522816287
43340000